RF to predict tti or log tti or ahd from all other data
 use households until find pop
dimensionality reduction?

factorsd impacting congestion, networks, tti, modeshares

tti website has other papers
modeshare variables
feature importance what is the direction of impact (negative vs. positive, impurity)
plot to show errors from rf optimization, 2d contour plot
rf/gbm as feature extractor
pca, condense and explain
kernel pca
update manuscript
final model (generalize additive model)
poster

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.inspection import permutation_importance

In [2]:
trf = pd.read_csv('../../data/tidy/tti-traffic19-renamed.csv')
net = pd.read_csv('../../data/tidy/data-aggregation-networkstats.csv')
inc = pd.read_csv('../../data/tidy/acs19-hh-income.csv')
mod = pd.read_csv('../../data/tidy/acs19-modeshares.csv')
unemp = pd.read_csv('../../data/tidy/acs19-unemp.csv')
pop = pd.read_csv('../../data/tidy/acs19-pop.csv')

In [3]:
trf.columns

Index(['msa', 'auto_commuters', 'annual_excess_fuel_consumed_commuters',
       'ahd_ph', 'tti', 'ahd_truck'],
      dtype='object')

In [4]:
net.columns

Index(['msa', 'n', 'm', 'k_avg', 'edge_length_total', 'edge_length_avg',
       'streets_per_node_avg', 'intersection_count', 'street_length_total',
       'street_segment_count', 'street_length_avg', 'circuity_avg',
       'self_loop_proportion', 'node_density_km', 'intersection_density_km',
       'edge_density_km', 'street_density_km', 'builing_area_sqkm',
       'area_sqkm'],
      dtype='object')

In [5]:
df = trf.merge(net, on='msa', how='left')

In [6]:
df.head()

,msa,auto_commuters,annual_excess_fuel_consumed_commuters,ahd_ph,tti,ahd_truck,n,m,k_avg,edge_length_total,...,street_segment_count,street_length_avg,circuity_avg,self_loop_proportion,node_density_km,intersection_density_km,edge_density_km,street_density_km,builing_area_sqkm,area_sqkm
0,"Abilene, TX",64,1282,2548,1.07,172,14357,39401,5.488751,1.716897e+07,...,20606,433.147439,1.067840,0.004465,2.012523,1.669790,2406.696840,1251.142264,7.048505,7133.829924
1,"Akron, OH",298,6741,15835,1.10,716,26865,70900,5.278243,1.519199e+07,...,36980,217.395754,1.056240,0.015089,11.231303,8.862967,6351.230011,3360.943838,14.307097,2391.975398
2,"Albany, GA",54,870,1905,1.08,96,8466,22838,5.395228,9.545102e+06,...,11954,417.399446,1.052833,0.002928,2.028965,1.667318,2287.583389,1195.808038,6.979580,4172.570196
3,"Albany-Lebanon, OR",35,465,1046,1.06,78,7132,17906,5.021312,8.280294e+06,...,9147,461.948061,1.150205,0.005466,1.193324,0.853498,1385.456359,706.999160,7.519705,5976.582645
4,"Albany-Schenectady-Troy, NY",297,6240,15617,1.15,732,36977,96867,5.239311,2.787244e+07,...,50900,285.961192,1.091565,0.033320,4.962653,4.060365,3740.737967,1953.471821,41.455466,7451.054339


In [7]:
df = df.merge(mod, left_on='msa', right_on='msa', how='inner')

In [8]:
df = df.merge(unemp, left_on = 'msa', right_on = 'msa', how = 'left')

In [9]:
df = df.merge(inc, left_on = 'msa', right_on = 'msa', how = 'left')

In [10]:
df = df.merge(pop, left_on = 'msa', right_on = 'msa', how = 'left')

In [11]:
df = df.dropna(subset=['tti'])
# df['log_tti'] = np.log10(df['tti'])

In [12]:
df.head()

,msa,auto_commuters,annual_excess_fuel_consumed_commuters,ahd_ph,tti,ahd_truck,n,m,k_avg,edge_length_total,...,ms_carp_per,ms_pt_per,ms_taxi_per,ms_bike_per,ms_walk_per,ms_wfh_per,unemp,med_hh_inc_dol,households,population
0,"Abilene, TX",64,1282,2548,1.07,172,14357,39401,5.488751,1.716897e+07,...,100.105752,0.510579,0.915432,0.265604,1.943037,5.499039,3.4,52511,60931,170669
1,"Akron, OH",298,6741,15835,1.10,716,26865,70900,5.278243,1.519199e+07,...,100.071737,1.303214,0.614118,0.096859,1.592914,4.545640,5.4,57285,286543,703845
2,"Albany, GA",54,870,1905,1.08,96,8466,22838,5.395228,9.545102e+06,...,100.114382,0.995359,0.906248,0.131145,1.585513,3.752774,10.6,44844,55714,148436
3,"Albany-Lebanon, OR",35,465,1046,1.06,78,7132,17906,5.021312,8.280294e+06,...,100.111969,0.170747,1.503313,0.645868,2.225274,4.960932,6.6,55893,47762,125048
4,"Albany-Schenectady-Troy, NY",297,6240,15617,1.15,732,36977,96867,5.239311,2.787244e+07,...,100.080156,3.618297,0.859834,0.280518,3.332137,4.555087,4.6,71285,352713,880736


In [13]:
len(df.columns)

35

In [14]:
len(df)

359

In [15]:
df = df.set_index('msa')

In [16]:
from pathlib import Path  
filepath = Path('../../data/tidy/congestion-exploration19.csv', index = False)  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)

In [17]:
rf = RandomForestRegressor(max_features= 8, n_estimators=100, random_state=0, max_depth = 9) #ccp_alpha=0.1
X = df.drop(columns=['tti'])
Y = (df['tti'])
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = .2, random_state = 1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
#y_pred


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
param_grid = { 
    'n_estimators': [50, 100, 200, 300, 400, ],
    'max_features': [5, 9, 12, 'sqrt', 'log2'],
    'max_depth' : [5,7, 9, 10, 12],   
}
CV_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 5)
CV_rf.fit(X_train, y_train)
CV_rf.best_params_

In [ ]:
mse = mean_squared_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'mse = {mse} mape = {mape} rmse = {rmse}')

In [ ]:
plt.scatter(y_test, y_pred)
plt.ylim(1, 1.5)
plt.xlim(1, 1.5)
x = np.linspace(0,2,20)
y = np.linspace(0,2,20)
plt.plot(x, y, color = 'red')
plt.savefig('../../figures/tti-pred')

In [ ]:
y_pred_train = rf.predict(X_train)
plt.scatter(y_train, y_pred_train)
plt.ylim(1, 1.5)
plt.xlim(1, 1.5)
x = np.linspace(0,2,20)
y = np.linspace(0,2,20)
plt.plot(x, y, color = 'red')

In [ ]:
result = permutation_importance(
    rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2)
tree_importances = pd.Series(result.importances_mean, index=X.columns)

In [ ]:
# sort features according to importance
sorted_idx = np.argsort(tree_importances)
pos = np.arange(sorted_idx.shape[0])

plt.figure(figsize=(8,9))
# plot feature importances
plt.barh(pos, tree_importances[sorted_idx], align="center")

plt.yticks(pos, np.array(X.columns)[sorted_idx])
plt.title("Feature Importance (MDI)")
plt.xlabel("Mean decrease in impurity");
plt.tight_layout()
plt.savefig('../../figures/tti-rf.png',dpi=300)

In [ ]:
rfc = RandomForestRegressor(max_features= 'log2', n_estimators=700, random_state=0, max_depth = 4 ) #ccp_alpha=0.1
A = df.drop(columns=['ahd_ph' ])
B = (df['ahd_ph'])
A_train, A_test, b_train, b_test = train_test_split(A, B, test_size = .2, random_state = 1)
rfc.fit(A_train, b_train)
b_pred = rfc.predict(A_test)
b_pred

In [ ]:
mse = mean_squared_error(b_test, b_pred)
mape = mean_absolute_percentage_error(b_test, b_pred)
rmse = np.sqrt(mean_squared_error(b_test, b_pred))
print(f'mse = {mse} mape = {mape} rmse = {rmse}')

In [ ]:
param_grid = { 
    'n_estimators': [100, 200,300, 400, 500, 600, 700],
    'max_features': [3, 5, 9, 12, 'sqrt', 'log2'],
    'max_depth' : [2, 3, 4,5,6,7,8, 9, 10, 11],   
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(A_train, b_train)
CV_rfc.best_params_

In [ ]:
plt.scatter(b_test, b_pred)
plt.ylim(0,40000)
plt.xlim(0, 40000)
x = np.linspace(0,40000,20)
y = np.linspace(0,40000,20)
plt.plot(x, y, color = 'red')
plt.savefig('../../figures/ahd_pred')

In [ ]:
result = permutation_importance(
    rfc, A_test, b_test, n_repeats=10, random_state=42, n_jobs=2)
tree_importances = pd.Series(result.importances_mean, index=X.columns)

In [ ]:
# sort features according to importance
sorted_idx = np.argsort(tree_importances)
pos = np.arange(sorted_idx.shape[0])

plt.figure(figsize=(8,9))
# plot feature importances
plt.barh(pos, tree_importances[sorted_idx], align="center")

plt.yticks(pos, np.array(A.columns)[sorted_idx])
plt.title("Feature Importance (MDI)")
plt.xlabel("Mean decrease in impurity");
plt.tight_layout()
plt.savefig('../../figures/ahd-rf.png',dpi=300)